In [1]:
import xlwings as xw
import numpy as np
import pandas as pd
import datetime as dt
import time
import sys

In [2]:
wb = xw.Book('重量计算流程_原始.xlsm')  #open the file with macro .xlsm

In [3]:
xw.sheets.add(name='Summary', before=wb.sheets[0])

ValueError: Sheet named 'Summary' already present in workbook

In [3]:
sht1 = wb.sheets[0]

In [9]:
sht1.range('A1').value = 'Air Way Bill GW'
sht1.range('A2').value = '包税 GW'
sht1.range('A3').value = '客人自有税号 GW'
sht1.range('A4').value = '包税修改后 GW'
sht1.range('A1:A4').color = (0, 225, 0)

In [ ]:
# sht1.range('A1:A4').clear_contents()

In [ ]:
rng = sht1.range('A1:A4')

In [ ]:
rng.autofit()

In [ ]:
insert_column = wb.macro('InsertColumn')

In [10]:
sht4 = wb.sheets[3]    #worksheet B

In [11]:
rng2 = sht4.range('L3:L143')

In [12]:
sht4.range('L3').select()  #select cell L3
#insert_column()             # insert 1 column

In [13]:
item_qty_rng = sht4.range('J3:J143')
item_wt_rng = sht4.range('Y3:Y143')     # some problem here, shouldn't be column Y? 
# because originall it's column x before added one more column later, then it becomes Y

In [14]:
sht4.range('L2').value = 'CALACULATED GW'

In [15]:
item_calc_wt = [a * b for a, b in zip(item_qty_rng.value, item_wt_rng.value)]

In [16]:
sht4.range('L3').options(transpose=True).value = item_calc_wt

In [17]:
sht4.range('L144').value = '=sum(L3:L143)'

In [18]:
sht1.range('B2').value = sht4.range('L144').value

In [19]:
adj_wt_item = (sht1.range('B1').value - sht1.range('B2').value) / len(sht4.range('L3:L143'))  # the weight need to adjust for each line

In [20]:
adj_wt_item

1.5752482269503538

In [21]:
adj_item_calc_wt = [round(v + adj_wt_item, 2) for v in item_calc_wt]  #adjusted weight

In [22]:
sht4.range('M3').select()
#insert_column()

In [23]:
sht4.range('M2').value = 'ADJUSTED GW'

In [24]:
sht4.range('m3').options(transpose=True).value = adj_item_calc_wt

In [25]:
sht4.range('M144').value = '=sum(M3:M143)'

In [26]:
diff = round(sht4.range('M144').value - sht4.range('K144').value, 2)

In [27]:
diff

0.67

In [ ]:
type(adj_item_calc_wt)

In [ ]:
index, value = max(zip(range(1000),adj_item_calc_wt), key=lambda item: item[1])  #find the maximum value of item in ADJUSTED GW

In [ ]:
index, value

In [28]:
rng2 = sht4.range('C3:C143')

In [29]:
item = rng2.value   # get the item list

In [30]:
from collections import Counter
dct = dict(Counter(item))  # find the occurrence of each item

In [31]:
sorted_by_value = sorted(dct.items(), key=lambda kv: kv[1]) #sorting by value

In [32]:
sorted_by_value

[('Clip', 1),
 ('plastic cup', 1),
 ('Hanger', 1),
 ('Armor', 1),
 ('Foot protection', 1),
 ('Arm guard', 1),
 ('Finger', 1),
 ('Stainless steel needle', 1),
 ('Foot cover', 1),
 ('Drying rack', 1),
 ('Wick', 1),
 ('Vest', 1),
 ('gloves', 1),
 ('Bracers', 1),
 ('Finger band', 1),
 ('Sling', 1),
 ('Mask', 1),
 ('humidifier', 1),
 ('wrench', 1),
 ('Air conditioning panel', 1),
 ('Exhaust gas cover', 1),
 ('exhaust pipe', 1),
 ('Motorcycle headlights', 1),
 ('Speaker Network', 1),
 ('Shift handle', 1),
 ('handle', 1),
 ('Shift knob', 1),
 ('nflator', 1),
 ('handlebar', 1),
 ('strap', 1),
 ('LED light', 1),
 ('Eyelash brush', 1),
 ('Shoehorn', 1),
 ('Agitator', 1),
 ('Table bar chair yarn', 1),
 ('Toe separation sleeve', 1),
 ('belt', 1),
 ('3D printer nozzles', 1),
 ('Leaf target', 2),
 ('vest', 2),
 ('Shoulderpad', 2),
 ('Buckle', 2),
 ('luggage rack', 2),
 ('handguard', 2),
 ('Decorative cover', 2),
 ('Storage Box', 2),
 ('Dustproof wheel cover', 2),
 ('Leggings', 2),
 ('eyebrow set', 2

In [33]:
for i in rng2:
    if i.value == sorted_by_value[0][0]:
        print(i.address, i.row, i.column)   # find the cell location of matching value
        row_num = i.row

$C$39 39 3


In [34]:
sht4.range(row_num, 13).value -= diff   # upto here, the Ajusted GW matches Air Way Bill GW